In [1]:
import sys
sys.executable

'/Users/jurriaan/opt/anaconda3/envs/conda_unosatenv/bin/python'

In [2]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
from os.path import join
import descartes
from PIL import Image

import fiona 
import pyproj
import time

import warnings
from importlib import reload

# see: https://github.com/karolzak/keras-unet
from keras_unet.models import satellite_unet 
from keras_unet.losses import jaccard_distance
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.utils import get_augmented
from keras.callbacks import ModelCheckpoint

import keras
from keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

warnings.simplefilter("ignore")

Using TensorFlow backend.


In [3]:
import load_data as ld
import preprocessing as pp
import prepare_for_network as pfn

In [4]:
# local paths, set your own path here
path_data_main = '/Volumes/other/datasets_and_ML/UNOSAT_Challenge'

# put your data in these dirs
path_data_local_train = join(path_data_main, 'Train_Dataset')
path_data_local_val = join(path_data_main, 'Validation_Dataset')
path_data_local_test = join(path_data_main, 'Evaluation_Dataset')

In [5]:
# other dirs are made automatically
dir_temp_data =  ld.make_dir(join(path_data_main, 'data_temp'))
dir_temp_data_train = ld.make_dir(join(dir_temp_data, 'train'))
dir_temp_data_test = ld.make_dir(join(dir_temp_data, 'test'))
dir_temp_plots = ld.make_dir(join(dir_temp_data, 'plots'))
dir_temp_labels = ld.make_dir(join(dir_temp_data, 'labels'))

In [6]:
#parameters
#size_sub_sample = (256, 256)
size_sub_sample = (512, 512)
inv_stride = 2

## Load and preprocess training and validation data

All files in 'Train_Dataset' are automatically selected for training.
All files in 'Validation_Dataset' are automatically selected for validation.

In [7]:
# settings preprocessing
quantile_clip_max = 0.999

In [8]:
dict_paths_train = ld.paths_in_dict(path_data_local_train)

dict_raster_layers_train, dict_data_train = pp.preprocess_data(dict_paths_train, dir_temp_plots, quantile_clip_max)
dict_labels = pp.get_label_data(dict_paths_train, dict_raster_layers_train)

list_paths_data_train = pp.save_stacked_arrays(dict_data_train, dir_temp_data_train)
list_paths_labels = pp.save_labels(dict_labels, dir_temp_labels)


#_, dict_data_test = pp.preprocess_data(dict_info_data_train, dir_temp_plots, quantile_clip_max)
#list_paths_data_test = pp.save_stacked_arrays(dict_data_test, dir_temp_data_train)

preprocessing...
Mosul
  winter vh
  winter vv
  spring vh
  spring vv
  summer vh
  summer vv
  autumn vh
  autumn vv
Najaf
  winter vh
  winter vv
  spring vh
  spring vv
  summer vh
  summer vv
  autumn vh
  autumn vv
Nasiryah
  winter vh
  winter vv
  spring vh
  spring vv
  summer vh
  summer vv
  autumn vh
  autumn vv
Souleimaniye
  winter vh
  winter vv
  spring vh
  spring vv
  summer vh
  summer vv
  autumn vh
  autumn vv


TypeError: get_label_data() missing 1 required positional argument: 'dir_save_plots'

In [ ]:
dict_info_data_val  = ld.paths_in_dict(dir_temp_data_val)

dict_raster_layers_val, dict_data_val = pp.preprocess_data(dict_info_data_val, dir_temp_plots, quantile_clip_max)
dict_labels_val = pp.get_label_data(dict_data, dict_raster_layers_train, dir_temp_labels)

list_paths_data_val = pp.save_stacked_arrays(dict_data_val, dir_temp_data_val)
list_paths_labels_val = pp.save_labels(dict_labels, dir_temp_labels_val)

In [ ]:
list_paths_data, list_paths_labels

## prepare data for network

In [ ]:
dir_temp_data_train_whole = ld.make_dir(join(dir_temp_data_train, 'whole'))
dir_temp_labels_whole = ld.make_dir(join(dir_temp_labels, 'whole'))

list_dir_np = [f for f in os.listdir(dir_temp_data_train_whole) if os.path.splitext(f)[1] == '.npy']
list_dir_np_label = [f for f in os.listdir(dir_temp_labels_whole) if os.path.splitext(f)[1] == '.npy']

for area_data_file, area_label_file in zip(list_dir_np, list_dir_np_label):
    print(area_data_file, area_label_file)
    pipeline_prepare_for_nn(area_data_file, dir_temp_data_train, inv_stride, size_sub_sample)
    pipeline_prepare_for_nn_labels(area_label_file, dir_temp_labels, inv_stride, size_sub_sample)


Mosul.npy Mosul.npy
(8, 10980, 10979)
new size padding 1: (8, 11264, 11264)
padding 2:
 dim_str: 
 new size array: (8, 11264, 11264)
 shape after split: (484, 8, 512, 512)
save_array_in_new_sub_dir
padding 2:
 dim_str: x
x1: (8, 11264, 11264)
x2: (8, 11776, 11264)
 new size array: (8, 11776, 11264)
x shape after split: (506, 8, 512, 512)
save_array_in_new_sub_dir
padding 2:
 dim_str: y
 new size array: (8, 11264, 11776)
y shape after split: (506, 8, 512, 512)
save_array_in_new_sub_dir
padding 2:
 dim_str: xy
x1: (8, 11264, 11264)
x2: (8, 11776, 11264)


## Deep learning

In [ ]:
test_run = True

In [ ]:
#size_sub_sample = (256, 256)
quantile_clip_max = 0.999
size_sub_sample = (512, 512)
inv_stride = 2
EPOCHS = 1
INIT_LR = 1e-3
BS = 32

IMAGE_DIMS =  size_sub_sample + (8,)
model = satellite_unet(IMAGE_DIMS)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

In [ ]:
print('load data...')
if test_run:
    # testing with one area
    trainX = np.load(file_X)
    trainY = np.load(file_Y)
else:
    # for real with all areas, big data
    trainX = ld.load_entire_dataset(join(dir_temp_data_train, 'split'))
    trainY = ld.load_entire_dataset(join(dir_temp_labels, 'split'))

In [ ]:
trainX = trainX.reshape(trainX.shape[0], trainX.shape[2], trainX.shape[3], trainX.shape[1])
trainY = trainY.reshape(trainY.shape[0], trainY.shape[1], trainY.shape[2], 1)

In [ ]:
train_datagen = get_augmented(
        trainX,
        trainY,
        X_val=None,
        Y_val=None,
        batch_size=BS,
        seed=0,
        data_gen_args=dict(
            rotation_range=90,
            height_shift_range=0,
            shear_range=0,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='constant'
        )
    )

In [ ]:
 model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  # metrics=["accuracy"],
                  metrics=[iou, iou_thresholded])

In [ ]:
filepath ="weights-improvement-{epoch:02d}-{iou:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='iou', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
H = model.fit_generator(generator=train_datagen,
                        steps_per_epoch=len(trainX) // BS,
                        epochs=EPOCHS,
                        callbacks=callbacks_list,
                        verbose=2
                        )

In [52]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_epochs.h5")

ValueError: cannot reshape array of size 1015021568 into shape (8,512,484)

In [ ]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["iou"], label="train_iou")

plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/iou")
plt.legend(loc="lower left")
plt.savefig("plot")
